In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="6"
import torch
import clip
from PIL import Image
from SoccerNet.utils import getListGames
import time
from tqdm import tqdm
from natsort import natsorted
import numpy as np

In [2]:
torch.cuda.is_available()

True

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Available CLIP models
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [5]:
# Use ResNet Model
# Load CLIP Models
model_dir = "/data/ted/hdsi/models"
model, preprocess = clip.load("RN50x64", device='cuda', download_root = model_dir)

In [6]:
def get_embedding(model, image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        
    return image_features

In [7]:
feat = get_embedding(model, "11.jpg")
feat.size()

torch.Size([1, 1024])

### Extract Features

In [8]:
games = getListGames("challenge")
# leagues = ["germany_bundesliga", "italy_serie-a", "spain_laliga"]
# games = [game for game in games if game.split("/")[0] in leagues]

In [9]:
len(games)

50

In [13]:
# Extract features of each image and save in npz format
start = time.time()

for game in games[:25]:
    for i in range(2):
        idx = i+1
        img_list_path = "challenge_sn_video/" + game + "/" + str(idx) + "_720p/"
        print(img_list_path)
        imgs = os.listdir(img_list_path)
        imgs = [item for item in imgs if ".jpg" in item]
        imgs = natsorted(imgs)
        
        j = 0 
        for img in imgs:
            if img == "CLIP_feats.npz":
                continue
            img_path = img_list_path+img
            feat = get_embedding(model, img_path)
            feat = feat.cpu().detach().numpy()
            if j == 0:
                features = feat
            else:
                features = np.vstack((features, feat))
            #os.remove(img_path)
            j = j+1
            
        np.savez_compressed(img_list_path+"CLIP_feats", features)
end = time.time()
print(end-start)

challenge_sn_video/england_epl/2016-2017/2017-05-13 - 18-00 Stoke City 1 - 4 Arsenal/1_720p/
challenge_sn_video/england_epl/2016-2017/2017-05-13 - 18-00 Stoke City 1 - 4 Arsenal/2_720p/
challenge_sn_video/england_epl/2016-2017/2017-04-30 - 18-00 Tottenham 2 - 0 Arsenal/1_720p/
challenge_sn_video/england_epl/2016-2017/2017-04-30 - 18-00 Tottenham 2 - 0 Arsenal/2_720p/
challenge_sn_video/england_epl/2016-2017/2017-04-10 - 18-00 Crystal Palace 3 - 0 Arsenal/1_720p/
challenge_sn_video/england_epl/2016-2017/2017-04-10 - 18-00 Crystal Palace 3 - 0 Arsenal/2_720p/
challenge_sn_video/england_epl/2016-2017/2017-04-02 - 18-00 Arsenal 2 - 2 Manchester City/1_720p/
challenge_sn_video/england_epl/2016-2017/2017-04-02 - 18-00 Arsenal 2 - 2 Manchester City/2_720p/
challenge_sn_video/england_epl/2016-2017/2017-03-18 - 18-00 West Brom 3 - 1 Arsenal/1_720p/
challenge_sn_video/england_epl/2016-2017/2017-03-18 - 18-00 West Brom 3 - 1 Arsenal/2_720p/
challenge_sn_video/england_epl/2016-2017/2017-02-04 - 18